# Data Initialization

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [0]:
Workspace.InitializeFrom(DataSource);

# Args

In [0]:
var args = new ImportArgs("CH", 2021, 3, default, default , default);

# Data Node

In [0]:
var dataNodeData = await Workspace.GetDataNodeDataReportParameters(args);

In [0]:
var dataNodeStates = await Workspace.GetDataNodeStateReportParameters(args);

In [0]:
var allYieldCurves = await Workspace.GetYieldCurveReportParameters(args);;

In [0]:
var singleDataNodeParameters = await Workspace.GetSingleDataNodeReportParameters(args);;

In [0]:
var interDataNodeParameters = await Workspace.GetInterDataNodeParameters(args);;

In [0]:
var partnerRatings = await Workspace.GetPartnerRatingsReportParameters(args);

In [0]:
var partnerDefaultRates = await Workspace.GetCreditDefaultRatesReportParameters(args);;

# Reports

In [0]:
await Report.ForObjects(dataNodeData)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.Portfolio).GroupRowsBy(x => x.DataNode)
            .ToTable()
            .ExecuteAsync()

In [0]:
await Report.ForObjects(dataNodeStates)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()
            

In [0]:
await Report.ForObjects(allYieldCurves)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.YieldCurveType).GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

In [0]:
await Report.ForObjects(singleDataNodeParameters)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

In [0]:
await Report.ForObjects(interDataNodeParameters)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.GroupOfContract).GroupRowsBy(x => x.LinkedDataNode)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

In [0]:
await Report.ForObjects(partnerRatings)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.Partner)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()

In [0]:
await Report.ForObjects(partnerDefaultRates)
            .WithQuerySource(Workspace)
            .GroupRowsBy(x => x.CreditRiskRating)
            .GroupColumnsBy(x => x.Period)
            .ToTable()
            .ExecuteAsync()